In [1]:
import pandas as pd
import numpy as np
import ast
import os
import cv2
from os import getcwd

In [2]:
result = pd.read_csv('result.csv', index_col=0)
da = pd.read_csv('DaycaresAnnotations_original.csv')
result['detection'] = 'daycare'
da['detection'] = 'daycare'

In [3]:
result.head(5)

,file,conf,x,y,xmin,ymin,xmax,ymax,width,height,detection
0,11926_20161008.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,daycare
1,11926_20170331.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,daycare
2,11946_20140215.jpg,0.597667,747.607422,422.410187,670.337044,339.748055,824.8778,505.072319,154.540756,165.324265,daycare
3,12645_20160829.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,daycare
4,12650_20120928.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,daycare


In [4]:
ground_truth = []
for r in range(len(da)):
    if da.loc[r, 'region_shape_attributes'] != "{}":
        filename = da.loc[r, 'filename']
        rect = ast.literal_eval(da.loc[r, 'region_shape_attributes'])
        # x-min
        xmin = int(rect['x'])
        # x-max
        xmax = int(rect['x'] + rect['width'])
        # y-min
        ymin = int(rect['y'])
        # y-max
        ymax = int(rect['y'] + rect['height'])
        detection = da.loc[r, 'detection']
        ground_truth.append([filename, detection, xmin, ymax, xmax, ymin])
        
ground_truth = pd.DataFrame(np.array(ground_truth), columns = ['file', 'detection', 'xmin', 'ymax', 'xmax', 'ymin'])
detection_result = result[['file', 'detection', 'conf', 'xmin', 'ymax', 'xmax', 'ymin']]
positive_detection_result = detection_result.loc[~((detection_result['xmin'] == 0) & (detection_result['xmax'] == 0))]
zero_detection_result = detection_result.loc[((detection_result['xmin'] == 0) & (detection_result['xmax'] == 0))]


In [5]:
positive_detection_files = positive_detection_result['file'].values
zero_detection_result_files = zero_detection_result['file'].values
relevant_ground_truth = ground_truth[ground_truth['file'].isin(positive_detection_files)].drop_duplicates()


In [6]:
print(len(relevant_ground_truth), len(positive_detection_result))
print(set(relevant_ground_truth['file'].values) - set(positive_detection_result['file'].values))  
assert len(relevant_ground_truth) == len(positive_detection_result)

297 297
set()


In [7]:
positive_detection_result[['xmin', 'ymax', 'xmax', 'ymin']] = positive_detection_result[['xmin', 'ymax', 'xmax', 'ymin']].astype(int)
positive_detection_result.sort_values(by=['file']).head(5)

/Users/muhammadmuhaimin/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,file,detection,conf,xmin,ymax,xmax,ymin
2,11946_20140215.jpg,daycare,0.597667,670,505,824,339
5,12753_20130505.jpg,daycare,0.948496,721,494,857,416
6,12753_20150725.jpg,daycare,0.962339,723,487,848,422
7,12753_20170712.jpg,daycare,0.763759,722,495,846,415
8,12753_20171004.jpg,daycare,0.667500,716,491,857,427


In [8]:
relevant_ground_truth.sort_values(by=['file']).head(5)

,file,detection,xmin,ymax,xmax,ymin
1542,11946_20140215.jpg,daycare,697,463,806,377
1564,12753_20130505.jpg,daycare,718,482,851,406
1569,12753_20150725.jpg,daycare,718,482,851,406
1577,12753_20170712.jpg,daycare,718,482,851,406
1579,12753_20171004.jpg,daycare,718,482,851,406


In [9]:
def put_files(df, loc):
    values = [[value[0], np.array2string(value[1:], separator=' ')
       .replace('[', '').replace(']', '').replace("'", "").replace("\n", "")
     ]for value in df.values]
    for value in values:
        filepath_label = loc + value[0]
        filepath_label = filepath_label.replace('jpg', 'txt')
#         print(value[1])
        with open(filepath_label, 'w') as f:
            f.write("%s\n" % value[1])
put_files(positive_detection_result, './mAP/input/detection-results/')
put_files(relevant_ground_truth, './mAP/input/ground-truth/')